# PlateShuttle Control Python Scripts
## Version. 1.1
### Written by Hiroki Noguchi, RIKEN

***

- Ver.1.0: Initial Version.
- Ver.1.1: Functions --> Class

***

## Class

In [1]:
class SBSPlateToByonoyTransV2Dev:
    
    def __init__(self, USBDevLoc, initPos, ByonoyInPos):
        self.USBDevLoc = USBDevLoc ## USB device path
        # zero position is x=0. You can select x= -1.0 ~ -190.0 (mm)
        self.initPos = initPos ## initPos (OT2 pipetting position), mm
        self.ByonoyInPos = ByonoyInPos ## ByonoyInPos (Load into Byonoy), mm
        
    
    # func: Connect grbl (ConnectSBSPlateToByonoyTransV2Dev)
    def connect(self):
        import serial
        import time
        print ('Connect SBS Plate To Byonoy Trans V2 Device ...')
        # global ser
        self.ser=serial.Serial(self.USBDevLoc, 115200)
        print ('Wakeup the device...')
        self.ser.write(b"\r\n\r\n")
        time.sleep(2)   # Wait for grbl to initialize 
        self.ser.write(b"$X\r\n\r\n") #Unlock grbl
        time.sleep(0.5)
        self.ser.flushInput()  # Flush startup text in serial input
        print("Wakeup the device done.")
    
    # func: dis-connect grbl
    def disconnect(self):
        self.ser.close()
        print("The SBS plate to byonoy trans v2 device is disconnected.")

    # func: homing
    def homing(self):
        import time
        import re
        print("Homing Cycle start...")
        self.ser.write(bytes("$H \n", 'UTF-8'))
        a=0
        while a == 0:
            line = self.ser.readline()
            line_str = str(line.decode('UTF-8'))
            if 'ok' in line_str:
                a=1
            time.sleep(0.1)
        print("-> Done.")

    # func: move
    def move(self, x):
        import time
        import re
        print("Move: " + "X," + str(x))
        self.ser.write(bytes("G0 " + "X " + str(x) + "\n", 'UTF-8'))
        line= self.ser.readline()
        a = 0
        while a == 0:
            line = self.ser.write(bytes("?", 'UTF-8'))
            time.sleep(0.1)
            line = self.ser.readline()
            line_list = re.findall('[0-9]{1,4}.[0-9]{3}', line.decode('UTF-8'))
            if float(line_list[0]) == -x:
                a=1
            time.sleep(0.02)
        print(" -> Done.")
    
    # func: move to initPos
    def pipettingPos(self):
        self.move(self.initPos)

    # func: move to ByonoyInPos
    def ByonoyPos(self):
        self.move(self.ByonoyInPos)

## Python Scripts (Step-by-Step, Manual)

In [5]:
# 0. Setting and call SBS plate to byonoy trans v2 dev class

theDevice = SBSPlateToByonoyTransV2Dev("/dev/cu.usbmodem11201", -2, -136)
print(theDevice.USBDevLoc)

/dev/cu.usbmodem11201


In [6]:
# 1. connect the device system.
theDevice.connect()

Connect SBS Plate To Byonoy Trans V2 Device ...
Wakeup the device...
Wakeup the device done.


In [7]:
# 2. Homing. it is needed when the system start.
theDevice.homing()

Homing Cycle start...
-> Done.


In [10]:
# 3-1. Move to initPos (OT2 pipetting position).
theDevice.pipettingPos()

Move: X,-2
 -> Done.


In [23]:
# 3-3. Move x mm.
theDevice.move(-190)

Move: X,-190
 -> Done.


In [9]:
# 3-2. ByonoyInPos (Load into Byonoy)
theDevice.ByonoyPos()

Move: X,-136
 -> Done.


In [11]:
# End. Dis-connect the device system.
theDevice.disconnect()

The SBS plate to byonoy trans v2 device is disconnected.


## Test Run

In [ ]:
# 0. Setting and call SBS plate to byonoy trans v2 dev class

theDevice = SBSPlateToByonoyTransV2Dev("/dev/cu.usbmodem1101", -2, -136)
print(theDevice.USBDevLoc)

# 1. connect the device system.
theDevice.connect()

# 2. Homing. it is needed when the system start.
theDevice.homing()


for i in range(1,101):
    print(f"Test: No,{i} **********")
    # 3-1. Move to initPos (OT2 pipetting position).
    theDevice.pipettingPos()
    # 3-2. ByonoyInPos (Load into Byonoy)
    theDevice.ByonoyPos()


theDevice.pipettingPos()
    
# End. Dis-connect the device system.
theDevice.disconnect()

#### 